# Problem 2 - Chemical Reactions
- In this problem you will study the dynamics of the reactions 
 - $ A \leftrightarrow B $
 - $ A \leftrightarrow B \leftrightarrow C \leftrightarrow A $
 
You will do this using a Monte Carlo approach that accounts for 'time' of reactions, and also using the rate matrix approach mentioned in the final class. 

Below, I will set up and run the Monte Carlo for the $A/B$ system, and you will have to generalize it for the $A/B/C$ system as well as do the Matrix Equation solution

In [ ]:
%pylab inline
from matplotlib import pylab as plt
import numpy as np

### Extra info about rate equations
Suppose you have $n$ different states in your system (like $n=2$ different types of molecules, $A$+$B$), then let $P$ represents the fraction in each state. So for example, for $n=2$, all $A$ would be $P=\left(
\begin{array}{c}
1\\
0\\
\end{array}
\right)$, and all $B$ would be
$\left( \begin{array}{c}
0\\
1\\
\end{array}
\right)$.

The equation describing the rate of change of this population vector is 
$\frac{dP}{dt} = W P$, where $W$ is a rate matrix

$ W=\left(\begin{array}{c c}
w_{11} w_{21}\\
w_{12} w_{22} \\
\end{array}
\right)$, 

and the solution to this can be solved as $P(t) = e^{W t} P(0)$.

In this matrix, the off-diagonal elements are the rate of transition between each state. It turns out the columns have to sum to $0$, so that condition gives the diagonal elements $w_{ii} = -\sum_{i\neq j} w_{ij}$

We will use this info later!

## Two-State Model Monte Carlo
### For a 2 State Model, i.e. a reaction between A and B, the energy landscape might look like the following
<img src="figures/2020/two-state.png" width="400px">

We can do a Monte Carlo simulation where we try to move from the current state to the other state every $dt$. Consider the situation, $k_{AB}=1/sec$ and $k_{BA}=0.05/sec$, and $dt = 0.01 sec$.

The probability of going from $A\rightarrow B$ if in $A$ in time $dt$ is $k_{AB}dt$
The probability of going from $B\rightarrow A$ if in $B$ in time $dt$ is $k_{BA}dt$

The Monte Carlo Algorithm is:
- based on current state, compute transition probability
- generate a random number $r$
 - if $r$ is less than transition probability, switch state
 - else, add current state to trajectory again

In [ ]:
k_ab = 1
k_ba = 0.05
dt = 0.01
seed = 1
np.random.seed(seed)

In [ ]:
# if in state 0 (A), what is the probability of switching
p_switch = {
    0: k_ab*dt,
    1: k_ba*dt,
}

n_steps = 500000

trajectory_list = []

starting_state = 0
trajectory = [starting_state]
mc_times = dt*np.arange(n_steps+1)
for i in range(n_steps):
    current_state = trajectory[-1]
    other_state = 1-current_state
    transition_prob = p_switch[current_state]
    if np.random.random() < transition_prob:
        trajectory.append(other_state)
    else:
        trajectory.append(current_state)
        
trajectory = np.array(trajectory)

In [ ]:
plt.plot(mc_times,trajectory)
#only show first part of trajectory
plt.xlim(0,1000)
plt.ylabel("state")
plt.xlabel('time')

prob_A = np.sum(trajectory==0)/len(trajectory)
prob_B = np.sum(trajectory==1)/len(trajectory)

print("Computed populations:")
print("\tA:",prob_A)
print("\tB:",prob_B)

### Problem 1.a)

Using just $k_{ab}$ and $k_{ba}$, derive the expected fraction of molecules / fraction of the time you would expect the system to be in state 0 ($A$), and in 1 ($B$) (hint, this is related to the equilibrium constant). 

** Type the derivation here so I know that you know where it comes from **

Compute these values in the cell below, and make sure they agree with the values above

### Problem 1.b) ###
Below, construction the matrix $W$ for this two state system. 

The code with use the function `expm` to exponentiate the Matrix and compute the populations of the system starting from all in state 0 ($A$).

Check that your final populations agree with the values above. You can add horizontal lines using `plt.axhline` to confirm that they go to these values for probability of $A$ and $B$ exactly.

In [ ]:
from scipy.linalg import expm

prob_vec = np.array((1,0))
populations = []
eval_times = np.arange(0,10,0.1)
for t in eval_times:
    populations.append( np.dot(expm(W*t),prob_vec) )
populations = np.array(populations)
    
plt.plot(eval_times,populations[:,0],label='$A$')
plt.plot(eval_times,populations[:,1],label='$B$')

#example axhline showing 50% in each population
plt.axhline(0.5,linestyle='--',color='k')

plt.legend(loc=0)
plt.xlabel("$t$")
plt.xlim(0,10)


### Problem 1c extra credit (only if you have time)
Compute and plot the autocorrelation function of the trajectory from Monte Carlo above. Also plot the theoretical curve we derived in class for this two state model to see if they agree.

## Problem 2
Consider the following 3 state model
<img src="figures/2020/three-state.png" width="500px">

a) Below, write the partition function for the three state model, using round numbers for the energies where the unit of energy is $k_B T=1$, i.e. $Z=e^{-E_A}+e^{-E_B}+e^{-E_C}$

b) Using this partition function, write the expected fraction of the time you expect the system to be in state A,B, and C, and also do this in code below so you have the numbers


c) 
 i. Construct the matrix $W$ for this problem, and compute the populations of the 3 state system vs time, starting with all the population in state $B$, using the exponential matrix code as above. Assume the rate $k_{AB}=1.0/sec$ .
  
  ii. Add horizontal lines with `plt.axhline` to show that your result from c.i matches your result from b
  
  It should look like the one below (but maybe not exactly)

<img src="figures/2020/populations_3state.jpg" width="400px">


d) 

Generalize the Monte Carlo code from above to work for the three state model. 

Hint:  you can use the rate matrix you already created to figure out the probabilities of switching in your Monte Carlo code.

i. As in my example above, show the state changeing from $0 \rightarrow 1 \rightarrow 2 $ with time

ii. Show that the probability of being in each of the states ends up being the value that you predicted in 2b